# DATA20001 Deep Learning - Group Project
## Image project

- Testing [resnext101_32x8d](https://pytorch.org/docs/stable/torchvision/models.html#id27) implementation of pretrained transfer learning models
- Data preprocessing includes data augmentation, shuffle and split train=17k, val=1.5k, test=1.5k
- Data augmentation IS applied
- No data balancing applied
- Model has a hidden layer



In [1]:
# automatically reload dependencies and repository content so that kernel need not be restarted
%load_ext autoreload
%autoreload 2

## Import dependencies and select correct device

In [2]:
# import dependencies
import train_model
import eval_model
import utils
from image_dataset import ImageDataset
from data_augmentation import DataAugmentation

import time
import torch
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

In [3]:
device = None
if torch.cuda.is_available():
    print("Using GPU")
    device = torch.device("cuda:0")
else:
    print("Using CPU")
    device = torch.device("cpu")

Using GPU


## Define datasets

- shuffle
- train: 17k
- valid: 1.5k
- test: 1.5k

In [4]:
mapping_file = "file_to_labels_table.csv"
df = pd.read_csv(mapping_file)
train_idx, val_idx = utils.get_train_val_indexes(df, 0.85, shuffle=True)
val_test_split_idx = int(val_idx.shape[0]*.5)
test_idx = val_idx[:val_test_split_idx]
val_idx = val_idx[val_test_split_idx:]

dataAugmentation = DataAugmentation()
trainDataset = ImageDataset(train_idx, dataAugmentation=dataAugmentation)
valDataset = ImageDataset(val_idx)
testDataset = ImageDataset(test_idx)

print("Saving test dataset temporarily so test can be performed after restarting kernel.")
utils.save_pickle(testDataset, "tmp_test_data_exp_transfer_learning_7.pkl")
print("# Train:", len(trainDataset))
print("# Valid:", len(valDataset))
print("# Test:", len(testDataset))

Saving test dataset temporarily so test can be performed after restarting kernel.
# Train: 17000
# Valid: 1500
# Test: 1500


## Define the model architecture

The initial transfer learning implementation

In [5]:
torch.cuda.empty_cache()

preTrainedModel = models.resnext101_32x8d(pretrained=True)

for param in preTrainedModel.parameters():
    param.requires_grad = False

num_ftrs = preTrainedModel.fc.in_features
num_classes = 14

hiddenLayerNeurons = 256

preTrainedModel.fc = torch.nn.Sequential(
    torch.nn.Linear(num_ftrs, hiddenLayerNeurons),
    torch.nn.ReLU(),
    torch.nn.Linear(hiddenLayerNeurons, num_classes),
    torch.nn.Sigmoid()
)

model = preTrainedModel.to(device)

## Define test case

Hyperparam search could be defined here. No hyperparam searches in this notebook as purpose of this test is to test the effect of data preprocessing technique on initial transfer learning model architecture and see how longer run on epochs affect loss.

**Test objective**:

- Testing [resnext101_32x8d](https://pytorch.org/docs/stable/torchvision/models.html#id27) implementation of pretrained transfer learning model
- Data preprocessing includes 
  - data augmentation, 
  - shuffle 
  - split train=17k, val=1.5k, test=1.5k
  - A hidden layer


**Training results**:

- Best validation results:
 - Loss: 0.1105	Accuracy: 0.9595
 - Precision: 0.7431 Recall: 0.7113
 - F1: 0.7269 Duration: 1m 36s
- Testing set results.
 - Accuracy	0.962
 - Precision	0.801
 - Recall 	0.677
 - F1-score	0.734

In [ ]:
n_epochs = 6

model, logs = train_model.train_model(
    model,
    trainDataset,
    valDataset,
    device,
    numberOfEpochs=n_epochs,
    returnLogs=True
)

## Save model and logs

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [8]:
pd.DataFrame(logs["val"]).to_csv("exp_transfer_learning_7_logs_val.csv", index=False)
pd.DataFrame(logs["train"]).to_csv("exp_transfer_learning_7_logs_train.csv", index=False)
torch.save(model, 'exp_transfer_learning_7_model.pkl')  # saves model with architecture

## Test model

- Load model back from state_dict and perform test on testing set
- Load test-set
- Load training time logs of performance

In [9]:
model = torch.load("exp_transfer_learning_7_model.pkl")
testDataset = utils.load_pickle("tmp_test_data_exp_transfer_learning_7.pkl")
logsTable = pd.read_csv("exp_transfer_learning_7_logs_val.csv")
y_hats, y_trues = eval_model.test_model(model, testDataset, device)

### Validation set performance during training

In [10]:
logsTable

,duration,loss,accuracy,precision,recall,f1,true positives,true negatives,false positives,false negatives
0,68.106851,0.701922,0.476143,0.065472,0.445912,0.114180,709,9290,10120,881
1,83.385108,0.108671,0.957190,0.750181,0.651572,0.697408,1036,19065,345,554
2,88.214685,0.108809,0.959000,0.795142,0.617610,0.695221,982,19157,253,608
3,95.511727,0.110485,0.959524,0.743101,0.711321,0.726864,1131,19019,391,459
4,78.332975,0.109100,0.959619,0.756925,0.687421,0.720501,1093,19059,351,497
5,84.746319,0.111016,0.960476,0.789193,0.652201,0.714187,1037,19133,277,553
6,80.511646,0.104946,0.960952,0.775788,0.681132,0.725385,1083,19097,313,507


### Testing set performance

In [11]:
print("Testing set results.")
print(f" Accuracy\t{round(eval_model.get_metric(y_trues, y_hats, 'accuracy'), 3)}")
print(f" Precision\t{round(eval_model.get_metric(y_trues, y_hats, 'precision'), 3)}")
print(f" Recall \t{round(eval_model.get_metric(y_trues, y_hats, 'recall'), 3)}")
print(f" F1-score\t{round(eval_model.get_metric(y_trues, y_hats, 'f1'), 3)}")

Testing set results.
 Accuracy	0.962
 Precision	0.801
 Recall 	0.677
 F1-score	0.734


### Extra analysis

How many positives and negatives has been predicted.

Tuples like: **(label, count)**

In [12]:
list(zip(*np.unique(y_hats, return_counts=True)))

[(0, 19643), (1, 1357)]

## Download test set

The testset will be made available during the last week before the deadline and can be downloaded in the same way as the training set.

## Predict for test set

You should return your predictions for the test set in a plain text file.  The text file contains one row for each test set image.  Each row contains a binary prediction for each label (separated by a single space), 1 if it's present in the image, and 0 if not. The order of the labels is as follows (alphabetic order of the label names):

    baby bird car clouds dog female flower male night people portrait river sea tree

An example row could like like this if your system predicts the presense of a bird and clouds:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0
    
The order of the rows should be according to the numeric order of the image numbers.  In the test set, this means that the first row refers to image `im20001.jpg`, the second to `im20002.jpg`, and so on.

If you have the prediction output matrix prepared in `y` you can use the following function to save it to a text file.

In [12]:
np.savetxt('results.txt', y, fmt='%d')

NameError: name 'y' is not defined